First, we need to define an LLM:

In [1]:
from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model_name="gemini-2.5-flash-lite")

Now, let's define our toy Pydantic model and let's assume we got an invalid output that can't parsed:

In [2]:
from langchain.output_parsers import RetryWithErrorOutputParser, PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field


class SearchAction(BaseModel):
    query: str = Field(description="A query to search for if a search action is taken")


parser = PydanticOutputParser(pydantic_object=SearchAction)

completion_with_error = "{'action': 'what is the weather like in Munich tomorrow}"

try:
    parser.parse(completion_with_error)
except Exception as e:
    print(e)

Invalid json output: {'action': 'what is the weather like in Munich tomorrow}
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


Let's define our output parser with retry (it's based on the original parser):

In [3]:
fix_parser = RetryWithErrorOutputParser.from_llm(llm=llm, parser=parser)

We can inspect the default retry prompt it's using:

In [4]:
from langchain.output_parsers.retry import NAIVE_RETRY_PROMPT

print(NAIVE_RETRY_PROMPT)

input_variables=['completion', 'prompt'] input_types={} partial_variables={} template='Prompt:\n{prompt}\nCompletion:\n{completion}\n\nAbove, the Completion did not satisfy the constraints given in the Prompt.\nPlease try again:'


The last step, we need to define a template for the prompt value to be substituted in the default prompt:

In [5]:
retry_template = (
    "Your previous response doesn't follow the required schema and fails parsing. Fix the response so that it follow the expected schema."
    "Do not change the nature of response, only adjust the schema."
    "\n\nEXPECTED SCHEMA:{schema}\n\n"
)
retry_prompt = PromptTemplate.from_template(retry_template)

Now, let's run our parser and we'll see that the output was fixed behind the scenes:

In [6]:
fixed_output = fix_parser.parse_with_prompt(
    completion=completion_with_error,
    prompt_value=retry_prompt.format_prompt(schema=parser.get_format_instructions()),
)

In [ ]:
fixed_output

SearchAction(query='what is the weather like in Munich tomorrow')

In [8]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"query": {"description": "A query to search for if a search action is taken", "title": "Query", "type": "string"}}, "required": ["query"]}\n```'